In [ ]:
import pandas as pd
import numpy as np
from pycirclize import Circos
from pycirclize.parser import Matrix
from pycirclize import config
from pycirclize.config import ann_adjust

In [ ]:
df = pd.read_csv('../data/cleaned/MergedFlow_2014-2022.csv')
pref = pd.read_csv('../data/cleaned/prefectures.csv')

In [ ]:
df = df[(df['Origin_id'] != 99) & (df['Destination_id'] != 99)]

In [ ]:
df

In [ ]:
columns2019 = ['Origin_id', 'Destination_id', 'Flow2019']  # Replace with actual column names
df2019 = df[columns2019]

In [ ]:
df2019 = df2019.rename(columns={'Flow2019': 'value'})

In [ ]:
pref = pref.drop('nam', axis=1)

In [ ]:
#Merge to replace 'source' with 'nam_jp'
df2019 = df2019.merge(pref, left_on='Origin_id', right_on='Origin_id', how='left').rename(columns={'nam_jp': 'source'})

#Merge to replace 'target' with 'nam_jp'
df2019 = df2019.merge(pref, left_on='Destination_id', right_on='Origin_id', how='left').rename(columns={'nam_jp': 'target'})

In [ ]:
#Drop unnecessary columns
df2019 = df2019.drop(columns=['Origin_id_x', 'Origin_id_y', 'Destination_id'])

In [ ]:
df2019.head()

In [ ]:
df2019['value'] = df2019['value'] * 1000

In [ ]:
df2019 = df2019[['source', 'target', 'value']]

In [ ]:
matrix = Matrix.parse_fromto_table(df2019)

In [ ]:
#Create sector names
sectors = list(set(df2019["source"]).union(set(df2019["target"])))

circos = Circos.chord_diagram(
    matrix,
    space=0.5,
    cmap="tab20",
    link_kws=dict(direction=1, ec="white", lw=0.05, alpha=0.7),
    label_kws=dict(size=7, r=120, color="white")
)

#Add annotations 
for sector in circos.sectors:
    track = sector.add_track((95, 100))  
    track.axis(fc="#EEEEEE", ec="none")
    
    # Use the middle of the sector for annotation placement
    middle_pos = (sector.start + sector.end) / 2

    #Annotate at the correct position
    track.annotate(
        middle_pos,  
        sector.name,
        label_size=7
    )

#Enable automatic annotation adjustment
ann_adjust.enable = True

fig = circos.plotfig()

In [ ]:
fig.savefig("../image/chord_flow_2019.png")